In [1]:
import tensorly as tl
from tensorly.decomposition import parafac, quantized_parafac
from tensorly.kruskal_tensor import kruskal_to_tensor, KruskalTensor
from tensorly.base import unfold
from tensorly.quantization import quantize_qint

import torch
tl.set_backend('pytorch')

# Example 1. Tensor quantization
Quantization scheme can be either affine or symmetric.

Scale and zero_point values to perform quantization are computed either per channel or per tensor (i.e. we get either vectors or scalars).

Thus, there are 4 types of quantization scheme:

    ``torch.per_tensor_affine``
    ``torch.per_tensor_symmetric``
    ``torch.per_channel_affine``
    ``torch.per_channel_symmetric``

##### Generate a random tensor

In [2]:
t = torch.randn(256, 256, 9)
print('||float_tensor|| = {}'.format(tl.norm(t)))

dtype = torch.qint8

||float_tensor|| = 767.418212890625


##### Per channel  quantization

In [3]:
for qscheme in [torch.per_channel_affine, torch.per_channel_symmetric]:
    print("\nPer channel quantization, dtype: {}, qscheme: {}".format(dtype, qscheme))
    
    for dim in range(len(t.shape)):
        qt, scale, zero_point = quantize_qint(t,\
                                              dtype,\
                                              qscheme,\
                                              dim = dim,\
                                              return_scale_zeropoint=True)

        print('Per dim {}, ||float_tensor - quant_tensor|| = {}'.format(dim, tl.norm(t - qt)))


Per channel quantization, dtype: torch.qint8, qscheme: torch.per_channel_affine
Per dim 0, ||float_tensor - quant_tensor|| = 7.239077568054199
Per dim 1, ||float_tensor - quant_tensor|| = 7.207361698150635
Per dim 2, ||float_tensor - quant_tensor|| = 7.412856578826904

Per channel quantization, dtype: torch.qint8, qscheme: torch.per_channel_symmetric
Per dim 0, ||float_tensor - quant_tensor|| = 7.239077568054199
Per dim 1, ||float_tensor - quant_tensor|| = 7.207361698150635
Per dim 2, ||float_tensor - quant_tensor|| = 7.412856578826904


##### Per tensor quantization

In [4]:
for qscheme in [torch.per_tensor_affine, torch.per_tensor_symmetric]:
    print("\nPer tensor quantization, dtype: {}, qscheme: {}".format(dtype, qscheme))

    
    qt, scale, zero_point = quantize_qint(t,\
                                          dtype,\
                                          qscheme,\
                                          dim = dim,\
                                          return_scale_zeropoint=True)
    print('Per tensor, ||float_tensor - quant_tensor|| = {}'.format(tl.norm(t - qt)))


Per tensor quantization, dtype: torch.qint8, qscheme: torch.per_tensor_affine
Per tensor, ||float_tensor - quant_tensor|| = 8.125895500183105

Per tensor quantization, dtype: torch.qint8, qscheme: torch.per_tensor_symmetric
Per tensor, ||float_tensor - quant_tensor|| = 8.125895500183105


# Example 2. Quantization of a tensor in Kruskal format:
    a) via quantization of the corresponding full tensor
    b) via quantization of decomposition factors.

##### Generate tensor 

In [5]:
rank = 16
shape = (64, 64, 9)

factors = [torch.randn((i, rank)) for i in shape] 
weights = torch.ones(rank)

# tensor in Kruscal format
krt = KruskalTensor((weights, factors))

# corresponding tensor in full format
t = kruskal_to_tensor(krt)

tnorm = tl.norm(t)
print('||float_factors||: {}'.format(tnorm))

||float_factors||: 711.4100341796875


##### Choose quantization scheme

In [6]:
dtype = torch.qint8

## Per tensor quantization
qscheme, dim = torch.per_tensor_affine, None

## Uncomment for per channel quantization
# qscheme, dim = torch.per_channel_affine, 0

##### a) Quantize the full tensor

In [7]:
t_quant = quantize_qint(t, dtype, qscheme, dim = dim)
print('||float_factors - float_factors_quantized|| = {}'.format(tl.norm(t - t_quant)))

||float_factors - float_factors_quantized|| = 10.898009300231934


##### b) Quantize several factors

In [8]:
num_factors = len(factors)
for num_quant_factors in range(1, num_factors + 1):
    
    qfactors = [quantize_qint(factors[i], dtype, qscheme, dim = dim)\
                for i in range(num_quant_factors)\
               ] + [factors[i] for i in range(num_quant_factors, num_factors)]

    qkrt = KruskalTensor((weights, qfactors))
    qt = kruskal_to_tensor(qkrt)
    print('\n[{}/{}] factors are quantized'.format(num_quant_factors, num_factors))
    print('||quant_factors - float_factors|| = {}'.format(tl.norm(qt - t)))
#     print('||quant_factors - float_factors_quantized|| = {}'.format(tl.norm(qt - t_quant)))

#     qt_quant = quantize_qint(qt, dtype, qscheme, dim = dim)
#     print('\nquant_factors_quantized - t_quant_factors: {}'.format(tl.norm(qt_quant - qt)/tnorm))
    
#     print('||quant_factors_quantized - float_factors|| = {}'.format(tl.norm(qt_quant - t)/tnorm))
#     print('||quant_factors_quantized - float_factors_quantized|| = {}'.format(tl.norm(qt_quant - t_quant)/tnorm))



[1/3] factors are quantized
||quant_factors - float_factors|| = 8.447911262512207

[2/3] factors are quantized
||quant_factors - float_factors|| = 14.076884269714355

[3/3] factors are quantized
||quant_factors - float_factors|| = 35.730384826660156


# Example 3. Quantized ALS
Compare standard ALS algorithm for finding  CP decomposition with its quantized version, when at the end of each ALS step approximated factor is quantized.

##### Generate tensor

In [9]:
rank = 8
shape = (128, 128, 9)

factors = [torch.randn((i, rank)) for i in shape] 
weights = torch.ones(rank)

# tensor in Kruscal format
krt = KruskalTensor((weights, factors))

# corresponding tensor in full format
t = kruskal_to_tensor(krt)

tnorm = tl.norm(t)
print('||float_factors||: {}'.format(tnorm))

||float_factors||: 1053.6529541015625


##### Find an approximation using ALS

In [38]:
normalize_factors = False
(factors_als, weights_als), _ =  quantized_parafac(t, rank, n_iter_max=50000,\
                                    init='random', tol=1e-8, svd = None,
                                    normalize_factors = normalize_factors)

In parafac original_tensor_norm = 1053.6529541015625
parafac has stopped after iteration 59


In [39]:
tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_als, factors_als))))

tensor(0.0003)

In [40]:
weights_als

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

##### Find an approximation using quantized ALS

In [41]:
dtype = torch.qint8

## Per tensor quantization
qscheme, dim = torch.per_tensor_affine, None

## Uncomment for per channel quantization
# qscheme, dim = torch.per_channel_affine, 0

In [42]:
normalize_factors = False
(factors_qals, weights_qals), _, scales, zero_points = quantized_parafac(
                                    t, rank, n_iter_max=1001,\
                                    init='random', tol= False, svd = None,\
                                    normalize_factors = normalize_factors,\
                                    qmodes = [0, 1, 2],
                                    warmup_iters = 60,
                                    quantize_every = 1,
                                    qscheme = qscheme, dtype = dtype, dim = dim,
                                    return_scale_zeropoint=True)

In parafac original_tensor_norm = 1053.6529541015625
iteration 500, diff_from_norm = 14.47594928741455, rel_rec_error = 0.013738820957188909
iteration 1000, diff_from_norm = 14.475943565368652, rel_rec_error = 0.013738815526514724


In [43]:
tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_qals, factors_qals))))

tensor(14.4759)

In [44]:
weights_qals

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [45]:
scales, zero_points

((tensor(0.0665), tensor(0.0308), tensor(0.0085)),
 (tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32)))

In [46]:
factors_qals[0]

tensor([[-1.1302,  1.6621, -4.0555,  ..., -1.7286, -0.8643, -1.9280],
        [ 3.3907, -0.9973,  1.7951,  ..., -3.1912,  3.5901, -2.5928],
        [-2.4599, -0.9308, -0.1330,  ..., -0.5983, -1.0637, -0.1995],
        ...,
        [ 2.1939, -1.0637,  2.7923,  ..., -1.4626,  0.0665, -5.3187],
        [ 1.1967, -0.3989, -3.5901,  ..., -2.5264, -1.1302,  1.7951],
        [ 1.0637,  0.3324, -3.3242,  ...,  1.3297,  2.9253,  1.3297]])

In [47]:
factors_qals[0]/scales[0]

tensor([[-17.,  25., -61.,  ..., -26., -13., -29.],
        [ 51., -15.,  27.,  ..., -48.,  54., -39.],
        [-37., -14.,  -2.,  ...,  -9., -16.,  -3.],
        ...,
        [ 33., -16.,  42.,  ..., -22.,   1., -80.],
        [ 18.,  -6., -54.,  ..., -38., -17.,  27.],
        [ 16.,   5., -50.,  ...,  20.,  44.,  20.]])

In [48]:
factors_qals[1]/scales[1]

tensor([[ 90.,   2.,   0.,  ...,  50.,  54.,   7.],
        [  7.,  16.,  13.,  ...,   5., -32.,  41.],
        [  3.,  13., -23.,  ...,  17.,  -6., -10.],
        ...,
        [ 90.,   2.,  -8.,  ...,  23.,  10., -28.],
        [-65., -41.,  13.,  ...,  44.,  31.,  -6.],
        [-31.,  17.,  -5.,  ...,  48., -26.,   3.]])

##### Try to quantize 2 of 3 factors

In [59]:
normalize_factors = False
(factors_qals, weights_qals), _, scales, zero_points = quantized_parafac(
                                    t, rank, n_iter_max=1001,\
                                    init='random', tol= False, svd = None,\
                                    normalize_factors = normalize_factors,\
                                    qmodes = [0, 1],
                                    warmup_iters = 60,
                                    quantize_every = 1,
                                    qscheme = qscheme, dtype = dtype, dim = dim,
                                    return_scale_zeropoint=True)

In parafac original_tensor_norm = 1053.6529541015625
iteration 500, diff_from_norm = 16.254718780517578, rel_rec_error = 0.015427013911214994
iteration 1000, diff_from_norm = 16.254718780517578, rel_rec_error = 0.015427013911214994


In [60]:
tl.norm(t - kruskal_to_tensor(KruskalTensor((weights_qals, factors_qals))))

tensor(16.2547)